In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #non-destructive copy
    alt_titles = {}
    alternate_title_keys = ['Also known as', 'Arabic', 'Cantonese', 'Chinese','French','Hangul','Hebrew','Hepburn',
                           'Japanese','Literally','Mandarin','McCune-Reischauer','Original title','Polish',
                           'Revised Romanization','Romanized','Russian','Simplified','Traditional','Yiddish']
    for key in alternate_title_keys:
        if key in movie:  # Add alternate title to dictionary and then remove from movie dict object
            alt_titles[key]=movie[key]
            movie.pop(key)
    
    if len(alt_titles)>0:
        movie['alt_titles']=alt_titles

    # merge column names that are similar together for consistency
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')          # old column name, new column name
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
        
    return movie

In [3]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wfile, kfile, rfile):
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kfile, low_memory=False)
    ratings = pd.read_csv(rfile)

    # Open and read the Wikipedia data JSON file.
    with open(wfile, mode='r') as file:
        wraw = json.load(file)
    
    # Write a list comprehension to filter out TV shows.
    wmovies = [movie for movie in wraw if 'No. of episodes' not in movie]

    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wmovies]

    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id']=wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df['imdb_id'].drop_duplicates()
    except Exception as e:
        print(e)

    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep=[col for col in wiki_movies_df.columns if wiki_movies_df[col].isnull().sum()<len(wiki_movies_df)*0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    # Create a variable that will hold the non-null values from the “Box office” column.
    box_office=wiki_movies_df['Box office'].dropna()
    
    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office=box_office.apply(lambda x:' '.join(x) if type(x)==list else x)
    # replace values listed as ranges with just single number
    box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

    # Write a regular expression to match the six elements of "form_one" of the box office data.
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'    

    # Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million, remove $ and million; convert to float; return value
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
            s = re.sub('\$|\s|[a-zA-z]', '', s)  # also takes care of US$ 
            value = float(s) * 10**6
            return value

        # if input is of the form $###.# billion, remove $ and billion; convert to float; return value
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
            s = re.sub('\$|\s|[a-zA-z]' , '', s)
            value = float(s) * 10**9
            return value

        # if input is of the form $###,###,###, remove $ and commas; convert to float; return value
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            s = re.sub('\$|,' , '', s)
            value = float(s)
            return value
        
        # Otherwise return NaN
        else:
            return np.nan
        
    # Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office']=box_office.str.extract(f'({form_one}|{form_two})',flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)  # drop old column with the string values
    
    # Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna().map(lambda x: ' '.join(x) if type(x)==list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)  # fix entries in the form of a range
    budget = budget.str.replace(r'\[\d+\]\s*', '', regex=True)
    wiki_movies_df['budget']=budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    #wiki_movies_df.drop('Budget', axis=1, inplace=True)

    # Clean the release date column in the wiki_movies_df DataFrame.
    release_date=wiki_movies_df['Release date'].dropna().apply(lambda x:' '.join(x) if type(x)==list else x)
    date_form1 = r'[a-zA-Z]+\s\d{1,2},\s\d{4}'  # Format: Month Day, Year
    date_form2 = r'\d{4}.\d{2}.\d{2}'   # Format: YYYY-MM-DD
    date_form3 = r'^[a-zA-Z]+\s\d{4}'  # Format: Month Year
    date_form4 = r'^\d{4}[\s\n]'  # Format: Year
    wiki_movies_df['release_date']=pd.to_datetime(release_date.str.extract(f'({date_form1}|{date_form2}|{date_form3}|{date_form4})')[0], \
        infer_datetime_format=True)
    #wiki_movies_df.drop('Release date', axis=1, inplace=True)

    # Clean the running time column in the wiki_movies_df DataFrame.
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m') # Convert h and m to the groups desired
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time']=running_time_extract.apply(lambda row: row[0]*60+row[1] if row[2]==0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    
    
     
    # 2. Clean the Kaggle metadata.
    # Per recommendation, only keep movies where 'adult' field is False. Then drop adult column since all will be false.
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True' #Convert video column to boolean.
    # Convert numeric columns. Use errors='raise' to be alerted to anything that doesn't convert.
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    # Convert string date to datetime
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

    
    # 3. Merged the two DataFrames into the movies DataFrame.
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])

    # 4. Drop unnecessary columns from the merged DataFrame.
    # drop title_wiki, release_date_wiki, Language, and production company(s) columns
    movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)

    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column]=df.apply(lambda row: row[wiki_column] if row[kaggle_column]==0 else row[kaggle_column], axis=1)
        df.drop(columns=wiki_column, inplace=True)

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data(movies_df, 'runtime','running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle','budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue','box_office')

    # 7. Filter the movies DataFrame for specific columns.
    movies_cols_to_keep = ['imdb_id','id','tagline', 'belongs_to_collection',  
        'url', 'year', 'imdb_link', 'Based on', 'Starring', 'Cinematography',
       'Country', 'Director', 'Distributor',
       'Editor(s)', 'Composer(s)', 'Producer(s)', 'Writer(s)', 
       'budget_kaggle', 'genres', 'homepage', 
       'original_language', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'release_date_kaggle', 'revenue', 'runtime', 'spoken_languages',
       'status', 'title_kaggle', 'vote_average',
       'vote_count']
    movies_df = movies_df[movies_cols_to_keep]

    # 8. Rename the columns in the movies DataFrame.
    cols_renamed={'budget_kaggle':'budget', 'id':'kaggle_id', 'title_kaggle':'title' }
    movies_df = movies_df.rename(columns=cols_renamed)
    
    # 9. Transform and merge the ratings DataFrame.
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1)
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
    
    return wiki_movies_df, movies_with_ratings_df, movies_df


In [4]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = 'data'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [5]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file, kaggle_file, ratings_file)

In [6]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [7]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Country,Language,...,Editor(s),Composer(s),Producer(s),Production company(s),Writer(s),imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",United States,English,...,Michael Tronick,"[Cliff Eidelman, Yello]","[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]",tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",United States,English,...,Howard E. Smith,Maurice Jarre,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",United States,"[English, Lao]",...,"[John Bloom, Lois Freeman-Fox]",Charles Gross,Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",United States,English,...,Susan E. Morse,NaN,Robert Greenhut,NaN,Woody Allen,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",US,English,...,David Stiven,Maurice Jarre,John Cornell,NaN,Paul Hogan,tt0099018,6939946.0,25000000.0,1990-12-19,95.0


In [8]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

,imdb_id,kaggle_id,tagline,belongs_to_collection,url,year,imdb_link,Based on,Starring,Cinematography,...,runtime,spoken_languages,status,title,vote_average,vote_count,rating_movieId,rating_rating,rating_count,rating_timestamp
0,tt0098987,9548,Kojak. Columbo. Dirty Harry. Wimps.,NaN,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Adventures of Ford Fairlane,6.2,72.0,1027.0,0.5,83.0,83.0
1,tt0098994,25501,All they risked was everything.,NaN,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,...,114.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"After Dark, My Sweet",6.5,17.0,2762.0,4.0,16035.0,16035.0
2,tt0099005,11856,The few. The proud. The totally insane.,NaN,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,...,112.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Air America,5.3,146.0,1278.0,5.0,4757.0,4757.0
3,tt0099012,8217,NaN,NaN,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,...,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Alice,6.3,57.0,886.0,4.5,18.0,18.0
4,tt0099018,25943,Who does he think he is?,NaN,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,...,95.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Almost an Angel,5.6,23.0,2807.0,5.0,23.0,23.0


In [9]:
# 15. Check the movies_df DataFrame. 
movies_df.head()

,imdb_id,kaggle_id,tagline,belongs_to_collection,url,year,imdb_link,Based on,Starring,Cinematography,...,production_companies,production_countries,release_date_kaggle,revenue,runtime,spoken_languages,status,title,vote_average,vote_count
0,tt0098987,9548,Kojak. Columbo. Dirty Harry. Wimps.,NaN,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,...,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1990-07-11,20423389.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Adventures of Ford Fairlane,6.2,72.0
1,tt0098994,25501,All they risked was everything.,NaN,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,...,"[{'name': 'Avenue Pictures Productions', 'id':...","[{'iso_3166_1': 'US', 'name': 'United States o...",1990-08-24,2700000.0,114.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"After Dark, My Sweet",6.5,17.0
2,tt0099005,11856,The few. The proud. The totally insane.,NaN,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,...,"[{'name': 'IndieProd Company Productions', 'id...","[{'iso_3166_1': 'US', 'name': 'United States o...",1990-08-10,33461269.0,112.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Air America,5.3,146.0
3,tt0099012,8217,NaN,NaN,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,...,"[{'name': 'Orion Pictures', 'id': 41}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1990-12-25,7331647.0,102.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Alice,6.3,57.0
4,tt0099018,25943,Who does he think he is?,NaN,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,...,"[{'name': 'Paramount Pictures', 'id': 4}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1990-12-21,6939946.0,95.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Almost an Angel,5.6,23.0
